In [1]:
import fitz  # PyMuPDF
import re
import nltk

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bigsl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bigsl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

CONTENT EXTRACTION

In [3]:
def extract_text_from_pdf(pdf_path):
    uber = fitz.open(pdf_path)
    google = fitz.open(pdf_path)
    tesla = fitz.open(pdf_path)
    text = ""
    for page_num in range(len([uber,google,tesla])):
        page = uber.load_page(page_num)
        page = google.load_page(page_num)
        page = tesla.load_page(page_num)
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("uber.pdf")
pdf_text = extract_text_from_pdf("google.pdf")
pdf_text = extract_text_from_pdf("tesla.pdf")

In [4]:
# Program to measure the similarity between 
# two sentences using cosine similarity. 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

# X = input("Enter first string: ").lower() 
# Y = input("Enter second string: ").lower() 
X ="believes"
Y ="expects"

# tokenization 
X_list = word_tokenize(X) 
Y_list = word_tokenize(Y) 

# sw contains the list of stopwords 
sw = stopwords.words('english') 
l1 =[];l2 =[] 

# remove stop words from the string 
X_set = {w for w in X_list if not w in sw} 
Y_set = {w for w in Y_list if not w in sw} 

# form a set containing keywords of both strings 
rvector = X_set.union(Y_set) 
for w in rvector: 
	if w in X_set: l1.append(1) # create a vector 
	else: l1.append(0) 
	if w in Y_set: l2.append(1) 
	else: l2.append(0) 
c = 0

# cosine formula 
for i in range(len(rvector)): 
		c+= l1[i]*l2[i] 
cosine = c / float((sum(l1)*sum(l2))**0.5) 
print("similarity: ", cosine) 



similarity:  0.0


RAG

In [6]:
# To load the default "document" dataset with 3 pdfs from document (index name is 'compressed' or 'exact')
from transformers import RagRetriever

retriever = RagRetriever.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base", dataset="document", index_name="compressed"
)

# To load your own indexed dataset built with the datasets library. More info on how to build the indexed dataset in examples/rag/use_own_knowledge_dataset.py
from transformers import RagRetriever

dataset = (
    ...
)  # dataset must be a datasets.Datasets object with columns "title", "text" and "embeddings", and it must have a faiss index
retriever = RagRetriever.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base", indexed_dataset=dataset)

# To load your own indexed dataset built with the datasets library that was saved on disk. More info in examples/rag/use_own_knowledge_dataset.py
from transformers import RagRetriever

dataset_path = "path/to/my/dataset"  # dataset saved via *dataset.save_to_disk(...)*
index_path = "path/to/my/index.faiss"  # faiss index saved via *dataset.get_index("embeddings").save(...)*
retriever = RagRetriever.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base",
    index_name="custom",
    passages_path=dataset_path,
    index_path=index_path,
)

# To load the legacy index built originally for Rag's paper
from transformers import RagRetriever

retriever = RagRetriever.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base", index_name="legacy")

You are using a model of type dpr to instantiate a model of type rag. This is not supported for all configurations of models and can yield errors.


AssertionError: Config has to be initialized with question_encoder and generator config

In [7]:
import spacy
import pytextrank

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("textrank")

example_text = """We, like other internet, technology, and media companies, are frequently subject to litigation based on allegations
of infringement or other violations of intellectual property rights, including patent, copyright, trade secrets, and
trademarks. Parties have also sought broad injunctive relief against us by filing claims in U.S. and international courts
and the U.S. International Trade Commission (ITC) for exclusion and cease-and-desist orders. In addition, patentholding companies may frequently seek to generate income from patents they have obtained by bringing claims
against us. As we continue to expand our business, the number of intellectual property claims against us has increased
and may continue to increase as we develop and acquire new products, services, and technologies.
Adverse results in any of these lawsuits may include awards of monetary damages, costly royalty or licensing
agreements (if licenses are available at all), or orders limiting our ability to sell our products and services in the U.S. or
elsewhere, including by preventing us from offering certain features, functionalities, products, or services in certain
jurisdictions. They may also cause us to change our business practices in ways that could result in a loss of revenues
for us and otherwise harm our business.
Many of our agreements with our customers and partners, including certain suppliers, require us to defend
against certain intellectual property infringement claims and in some cases indemnify them for certain intellectual
property infringement claims against them, which could result in increased costs for defending such claims or
significant damages if there were an adverse ruling in any such claims. Such customers and partners may also
discontinue the use of our products, services, and technologies, as a result of injunctions or otherwise, which could
result in loss of revenues and harm our business. Moreover, intellectual property indemnities provided to us by our
suppliers, when obtainable, may not cover all damages and losses suffered by us and our customers arising from
intellectual property infringement claims. Furthermore, in connection with our divestitures, we have agreed, and may in
the future agree, to provide indemnification for certain potential liabilities, including those associated with intellectual
property claims. Regardless of their merits, intellectual property claims are often time consuming and expensive to
litigate or settle. To the extent such claims are successful, they could harm our business, including our product and
service offerings, financial condition, and operating results."""
print('Original Document Size:',len(example_text))
doc = nlp(example_text)

for sent in doc._.textrank.summary(limit_phrases=2, limit_sentences=2):
	print(sent)
	print('Summary Length:',len(sent))



ModuleNotFoundError: No module named 'spacy'

USER INTERFACE

In [8]:
from flask import Flask, request, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_files():
    uploaded_files = request.files.getlist("files")
    # Process files, extract text, compare, and generate insights
    return render_template('results.html', results=results)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

D:\New folder (2)\New folder\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
